# line 47 & 48: pair discovery and trading period hardcoded - line 74: threshold is 2 SD
Line 93: Why divide by 2? We still haven't concluded short and long are equaly likely.
Line 141: Why use cumilative returns? Shoulnd't it be just returns?
Line 154: By adding top5 and top20, we are making duplicates of the top 5. We should only consider the bigger set here (top20)

In [9]:
import ccxt
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from itertools import combinations
from tabulate import tabulate

# Initialize Binance API
binance = ccxt.binance()

# Define the cryptocurrency symbols that are supported on Binance
crypto_symbols = [
    'BTC/USDT', 'ETH/USDT', 'BNB/USDT', 'XRP/USDT', 'ADA/USDT', 'SOL/USDT', 'DOGE/USDT', 'DOT/USDT',
    'LTC/USDT', 'AVAX/USDT', 'LINK/USDT', 'XLM/USDT', 'UNI/USDT', 'BCH/USDT', 'MATIC/USDT', 'LUNA/USDT',
    'VET/USDT', 'ETC/USDT', 'FIL/USDT', 'USDC/USDT', 'TRX/USDT', 'ATOM/USDT', 'ALGO/USDT', 'MANA/USDT',
    'XTZ/USDT', 'AAVE/USDT', 'EGLD/USDT', 'THETA/USDT', 'HBAR/USDT', 'SUSHI/USDT', 'KSM/USDT', 'CAKE/USDT',
    'NEAR/USDT', 'AXS/USDT', 'FTM/USDT', 'SAND/USDT', 'ENJ/USDT', 'GALA/USDT', 'ONE/USDT', 'ZIL/USDT',
    'WAVES/USDT', 'DASH/USDT', 'MKR/USDT', 'COMP/USDT', 'BAT/USDT', 'CHZ/USDT', 'HOT/USDT', 'QTUM/USDT',
    'LRC/USDT', 'CELO/USDT', 'OCEAN/USDT', 'REN/USDT', 'ZRX/USDT', 'RSR/USDT', 'BAL/USDT', 'SRM/USDT',
    'AR/USDT', '1INCH/USDT', 'SKL/USDT', 'KAVA/USDT', 'INJ/USDT', 'CVC/USDT', 'DENT/USDT', 'ICX/USDT',
    'SC/USDT', 'NANO/USDT', 'SNT/USDT', 'OMG/USDT', 'ANT/USDT', 'STORJ/USDT', 'GLM/USDT', 'KNC/USDT',
    'FET/USDT', 'POWR/USDT', 'BAND/USDT', 'OGN/USDT', 'UTK/USDT', 'WAN/USDT', 'ARK/USDT', 'SYS/USDT',
    'IOTX/USDT', 'UST/USDT', 'BSV/USDT', 'XEM/USDT', 'ZEN/USDT', 'DGB/USDT', 'XVG/USDT', 'COTI/USDT',
    'ORBS/USDT', 'ALICE/USDT', 'CTK/USDT', 'DODO/USDT', 'FLM/USDT', 'HNT/USDT', 'ROSE/USDT', 'TWT/USDT',
    'YFI/USDT', 'ZEC/USDT', 'STMX/USDT', 'LIT/USDT', 'CHR/USDT', 'SXP/USDT', 'AKRO/USDT', 'REP/USDT',
    'NKN/USDT', 'MTL/USDT'
]

# Fetch close prices function
def fetch_close_prices(symbols, start, end):
    close_prices = {}
    for symbol in symbols:
        try:
            ohlcv = binance.fetch_ohlcv(symbol, timeframe='1d', since=start, limit=1000)
            # Convert to pandas DataFrame
            df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
            df.set_index('timestamp', inplace=True)
            # Filter the DataFrame to only include data up to the end date
            df = df[df.index <= pd.to_datetime(end, unit='ms')]
            close_prices[symbol] = df['close']
        except Exception as e:
            print(f"Error fetching data for {symbol}: {e}")
    return pd.DataFrame(close_prices)

# Define the rolling window parameters
formation_period_months = 12
trading_period_months = 3
end_date = datetime.strptime('2022-06-01', '%Y-%m-%d')
start_date = datetime.strptime('2021-01-01', '%Y-%m-%d')

# Convert date to timestamp
start_timestamp = int(start_date.timestamp() * 1000)
end_timestamp = int(end_date.timestamp() * 1000)

# Function to preprocess data
def preprocess_data(data):
    data = data.resample('D').ffill()  # Forward fill missing days
    if data.isna().any().any():
        return None  # Exclude cryptos with missing data
    data['return'] = data.pct_change().fillna(0)
    data['cumulative_return'] = (1 + data['return']).cumprod()
    return data

# Function to normalize price series
def normalize_series(series):
    return (series - series.mean()) / series.std()

# Function to calculate sum of squared deviations
def calculate_ssd(series1, series2):
    return np.sum((series1 - series2) ** 2)

# Function to implement trading strategy
def trade_pairs(pair, data, threshold=2):
    symbol1, symbol2 = pair[:2]  # Unpack only the first two elements
    series1 = normalize_series(data[symbol1]['cumulative_return'])
    series2 = normalize_series(data[symbol2]['cumulative_return'])
    spread = series1 - series2
    mean_spread = spread.mean()
    std_spread = spread.std()
    
    # Trading signals
    long_signals = (spread < mean_spread - threshold * std_spread).astype(int)
    short_signals = (spread > mean_spread + threshold * std_spread).astype(int)
    
    # Positions
    long_positions = long_signals.shift(1).fillna(0)
    short_positions = short_signals.shift(1).fillna(0)
    
    # Returns
    returns1 = data[symbol1]['return']
    returns2 = data[symbol2]['return']
    strategy_returns = (long_positions * returns2 - short_positions * returns1) / 2
    
    # Ensure index alignment
    strategy_returns = strategy_returns.reindex(data[symbol1].index).fillna(0)
    
    return strategy_returns

# Rolling window approach for pairs trading
def rolling_pairs_trading(symbols, start_date, end_date, formation_period_months, trading_period_months):
    all_stats = []
    current_start_date = start_date

    while current_start_date < end_date:
        formation_end_date = current_start_date + timedelta(days=formation_period_months * 30)
        trading_end_date = formation_end_date + timedelta(days=trading_period_months * 30)

        if trading_end_date > end_date:
            break
    
        # Fetch close prices for the formation period
        formation_close_prices = fetch_close_prices(symbols, int(current_start_date.timestamp() * 1000), int(formation_end_date.timestamp() * 1000))
        formation_close_prices = formation_close_prices.dropna(axis=1)  # Remove columns with any missing values
        print(f"Formation period from {current_start_date} to {formation_end_date}:")
        print(formation_close_prices.head())
    
        # Fetch close prices for the trading period
        trading_close_prices = fetch_close_prices(symbols, int(formation_end_date.timestamp() * 1000), int(trading_end_date.timestamp() * 1000))
        trading_close_prices = trading_close_prices[formation_close_prices.columns]  # Keep only the columns that were valid in the formation period
        print(f"Trading period from {formation_end_date} to {trading_end_date}:")
        print(trading_close_prices.head())

        # Preprocess data
        formation_preprocessed_data = {symbol: preprocess_data(formation_close_prices[[symbol]].dropna()) for symbol in formation_close_prices.columns}
        trading_preprocessed_data = {symbol: preprocess_data(trading_close_prices[[symbol]].dropna()) for symbol in trading_close_prices.columns}

        # Remove any symbols with None values
        formation_preprocessed_data = {k: v for k, v in formation_preprocessed_data.items() if v is not None}
        trading_preprocessed_data = {k: v for k, v in trading_preprocessed_data.items() if v is not None}

        # Check if there are any valid symbols left after preprocessing
        if not formation_preprocessed_data or not trading_preprocessed_data:
            print("No valid symbols after preprocessing. Skipping this window.")
            current_start_date = formation_end_date
            continue

        # Form pairs in the formation period
        pairs = []
        for symbol1, symbol2 in combinations(formation_preprocessed_data.keys(), 2):
            series1 = normalize_series(formation_preprocessed_data[symbol1]['cumulative_return'])
            series2 = normalize_series(formation_preprocessed_data[symbol2]['cumulative_return'])
            ssd = calculate_ssd(series1, series2)
            pairs.append((symbol1, symbol2, ssd))

        # Sort pairs by sum of squared deviations
        pairs.sort(key=lambda x: x[2])

        # Select top 5, top 20
        top_5_pairs = pairs[:5]
        top_20_pairs = pairs[:20]

        # Calculate excess returns for selected pairs during the trading period
        selected_pairs = top_5_pairs + top_20_pairs
        excess_returns = pd.DataFrame(index=trading_preprocessed_data[list(trading_preprocessed_data.keys())[0]].index)

        for pair in selected_pairs:
            strategy_returns = trade_pairs(pair, trading_preprocessed_data)
            # Align the index of strategy_returns with excess_returns
            strategy_returns = strategy_returns.reindex(excess_returns.index).fillna(0)
            excess_returns[f'{pair[0]}_{pair[1]}'] = strategy_returns

        # Calculate trading statistics
        def trading_statistics(excess_returns):
            stats = {
                'avg_price_deviation_trigger': excess_returns.apply(lambda x: (x > 0).mean(), axis=0).mean(),
                'avg_pairs_traded_per_period': excess_returns.apply(lambda x: (x != 0).sum(), axis=1).mean(),
                'avg_round_trip_trades_per_pair': excess_returns.apply(lambda x: (x != 0).sum(), axis=0).mean(),
                'std_round_trip_trades_per_pair': excess_returns.apply(lambda x: (x != 0).sum(), axis=0).std(),
                'avg_time_pairs_open_months': excess_returns.apply(lambda x: (x != 0).sum() / len(x), axis=0).mean(),
                'std_time_pairs_open_months': excess_returns.apply(lambda x: (x != 0).sum() / len(x), axis=0).std()
            }
            return stats

        # Calculate trading statistics for selected pairs
        top_5_excess_returns = excess_returns[[f'{pair[0]}_{pair[1]}' for pair in top_5_pairs]]
        top_20_excess_returns = excess_returns[[f'{pair[0]}_{pair[1]}' for pair in top_20_pairs]]

        top_5_stats = trading_statistics(top_5_excess_returns)
        top_20_stats = trading_statistics(top_20_excess_returns)
        portfolio_stats = trading_statistics(excess_returns)

        # Compile results into a DataFrame
        summary_df = pd.DataFrame({
            'Top 5': top_5_stats,
            'Top 20': top_20_stats,
            'All Pairs': portfolio_stats
        })

        all_stats.append(summary_df)

        # Move the window forward
        current_start_date = formation_end_date

    return all_stats


In [10]:
# Run the rolling pairs trading strategy
all_stats = rolling_pairs_trading(crypto_symbols, start_date, end_date, formation_period_months, trading_period_months)

# Check if there are any statistics collected
if not all_stats:
    print("No statistics collected. Please check the date ranges and data availability.")
else:
    # Display the summary statistics for each window
    for i, stats in enumerate(all_stats):
        print(f"Window {i + 1}")
        print(stats)
        print()

    # Display the summary statistics in a table format
    def display_summary_table(summary_df):
        panel_a = summary_df.loc[['avg_price_deviation_trigger', 'avg_pairs_traded_per_period', 'avg_round_trip_trades_per_pair',
                                  'std_round_trip_trades_per_pair', 'avg_time_pairs_open_months', 'std_time_pairs_open_months']]
        
        headers = ['Statistic', 'Top 5', 'Top 20', 'All Pairs']
        
        print("Panel A: Trading Statistics")
        print(tabulate(panel_a.reset_index().values, headers=headers, tablefmt='grid'))
        
        # Panel B: Pair Portfolio Composition is omitted as it does not directly apply to crypto pairs

    # Display the summary table for the last window
    display_summary_table(all_stats[-1])

Formation period from 2021-01-01 00:00:00 to 2021-12-27 00:00:00:
            BTC/USDT  ETH/USDT  BNB/USDT  XRP/USDT  ADA/USDT  SOL/USDT  \
timestamp                                                                
2021-01-01  29331.69    728.91   37.7762   0.23746   0.17509    1.8421   
2021-01-02  32178.33    774.56   38.2331   0.22064   0.17742    1.7999   
2021-01-03  33000.05    978.28   41.2575   0.22540   0.20615    2.1779   
2021-01-04  31988.71   1041.43   41.1333   0.23565   0.22528    2.4909   
2021-01-05  33949.53   1099.56   41.8219   0.22573   0.25873    2.1636   

            DOGE/USDT  DOT/USDT  LTC/USDT  AVAX/USDT  ...  ROSE/USDT  \
timestamp                                             ...              
2021-01-01   0.005680    8.2776    126.24     3.6497  ...    0.04210   
2021-01-02   0.010526    9.2037    136.96     3.4956  ...    0.04095   
2021-01-03   0.009821   10.1307    161.24     3.4691  ...    0.04129   
2021-01-04   0.009761    9.5126    155.34     3.5999  .